In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option("Display.max_columns", None)
# Example: pd.set_option('display.max_columns', None)

In [2]:
import datetime
from dateutil.relativedelta import relativedelta

def get_week_start_dates(start_date, end_date):
    week_start_dates = []
    current_date = start_date
    while current_date <= end_date:
        week_start_date = current_date - datetime.timedelta(days=current_date.weekday())
        week_start_dates.append(week_start_date)
        current_date += relativedelta(weeks=1)
    return week_start_dates

start_date = datetime.date(2024, 8, 26)
end_date = datetime.date(2024, 9, 8)

week_start_dates = get_week_start_dates(start_date, end_date)
print(week_start_dates)

[datetime.date(2024, 8, 26), datetime.date(2024, 9, 2)]


In [8]:
%%time
res =  pd.DataFrame()
for date in week_start_dates:
    query = f"""with 
    all_cust   --- To get the total registered customer till the cut off date
    AS( 
        SELECT count(distinct cust_id) Total_registered_users
        from prj-prod-dataplatform.dl_customers_db_raw.tdbk_customer_mtb
        where created_dt <= "{date}" and cust_id is not null 
        ) ,
    open_TSA_cust --- TSA customer which was opened before the cut off date and closed date is either null or greater than cutoff date
    AS(
        SELECT count (distinct OFCUSTOMERID) Total_open_TSA_customers from `core_raw.customer_accounts` 
            where CRINTERDESC like 'Transactional Savings Account Inv_R'
            and OFCUSTOMERID in (select distinct cust_id 
            from prj-prod-dataplatform.dl_customers_db_raw.tdbk_customer_mtb where cust_id is not null)
            AND OFDATEOPENED <= "{date}"
            AND (OFDATECLOSED = DATE('1970-01-01') OR  OFDATECLOSED > "{date}")
            )           
            ,
    loan_cust --list of customers having loan either in status Normal or Arrears at the cut off date
    AS (
        SELECT 
            distinct CAST(lmt.customerId AS STRING) customerId
        FROM 
            `prj-prod-dataplatform.risk_credit_mis.loan_master_table` lmt
            inner join prj-prod-dataplatform.risk_credit_mis.loan_bucket_flow_report_core lbfrc on lbfrc.loanAccountNumber = lmt.loanAccountNumber
        WHERE lbfrc.bucketDate = date_sub(date("{date}"), interval 1 day)  --- need to change the date
        and lbfrc.loanStatus IN ('Normal', 'In Arrears')
            )           
            ,
    ACL_Customer ---- ACL customer are TSBL loans with sub product type FP and disbursement date <= cutoff date
        as 
            (select lmt.customerId FROM `prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` a
                inner join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = a.digitalLoanAccountId
                where coalesce(a.loanType, 'NA') = 'TSBL'and coalesce(a.loan_sub_product_type, 'NA') = 'FP'
                and date_trunc(a.loanDigitalAppliedDateAndTime, day) <= "{date}"
                and date_trunc(lmt.disbursementDateTime, day) <= "{date}" --- need to change the date
                ),
    non_ACL_cust --list of all customers who have no ACL loans (loatype TSBL and sub-product type FP)
    AS (
        SELECT 
            distinct CAST(cust_id AS STRING) customerId
        FROM 
            `prj-prod-dataplatform.dl_customers_db_raw.tdbk_customer_mtb`
        WHERE 
            cast(cust_id as string) not in (select customerId from loan_cust)
            and created_dt <= "{date}" and cust_id is not null 
    )
    /*So, new definition of MAU is. Customer with open TSA and at least one any transaction for the amount greater than 100 PHP in the last 30 days. 
    No active loan. OR has balance of the TSA + any Stash greater than 100 PHP. Whoch means if he didnt transact but he has money on TSA - he is MAU. 
    If he doesnt transact and doesnt have money on TSA but has more than 100 PHP on Stash - he is active */
    ,        
    MAU_with_trx 
    AS (
        SELECT DISTINCT customer_id 
        FROM `prj-prod-dataplatform.risk_mart.customer_transactions`
        WHERE transaction_date BETWEEN DATE_SUB(date("{date}"), INTERVAL 30 DAY) AND date("{date}")
        AND ABS(trx_amount) > 100  --- transaction greater than 100
        AND account_type = "Tonik Account"  -- TSA account type
        AND customer_id NOT IN (SELECT customerId FROM loan_cust)  --- no active loan
        )
        ,
    MAU_with_bal 
     AS (
        SELECT 
            DISTINCT client_id 
        FROM 
            `prj-prod-dataplatform.risk_mart.customer_balance`
        WHERE
            clearedbalance > 100  --- balance in any account greater than 100
            AND account_type IN ("Tonik Account","Group Stash","Individual Stash")  ---- TSA, Group Stash and Individual Stash
            AND balanceDateAsOf between date_sub(date("{date}"), interval 30 day) and date("{date}") ---- in last 30 days
            AND client_id NOT IN (SELECT customerId FROM loan_cust) 
        )            ,
    MAU --- 
        AS (
        SELECT COUNT(distinct customer_id) AS MAU 
        FROM 
            (SELECT customer_id 
            FROM 
                MAU_with_trx
            UNION DISTINCT
            SELECT 
                client_id AS customer_id
            FROM 
                MAU_with_bal)),
    /* a) MMAU should be without active SIL, Quck and Flex loan. but when they will have ACL TSA loan - those should be included into MMAUs. 
    b)  I would like to ask you to add  
    1) the active TSA ACL loans issued (today its zero) after the MMAU.  
    2) Difference between MMAU with TSA aCL loan and TSA ACL LOans that will show us the current pool of eligible customers. 
    3) Offers extended. They should live somewhere on the datalake table. 
    For example this week we will issue 1000 offers. Next week we will issue 10 000 more. So netx week it will be 11 k offers sent.   
    And last one 4) Uptake. TSA ACL Loans active divided into Offers sent. as %.   */
    MMAU_customerbase as (select cast(customerId as string)customerId from non_ACL_cust union all (select cast(customerId as string)customerId from ACL_Customer)) , 
    transactional_cust --Mareks criteria
    AS (
        SELECT 
            DISTINCT customer_id
        FROM 
            `prj-prod-dataplatform.risk_mart.customer_transactions`
        WHERE
            transaction_date BETWEEN DATE_SUB(date("{date}"), INTERVAL 3 MONTH) AND date("{date}")
            AND customer_id in (SELECT customerId FROM MMAU_customerbase)
        GROUP BY
            customer_id
        HAVING 
            COUNT(*) >= 5
    ),
    cust_with_balance --Mareks criteria
    AS (
        SELECT 
            DISTINCT client_id AS customer_id 
        FROM 
            `prj-prod-dataplatform.risk_mart.customer_balance`
        WHERE
            clearedbalance > 1000
            AND account_type IN ("Tonik Account","Group Stash","Individual Stash")
            AND balanceDateAsOf between date_sub(date("{date}"), interval 30 day) and date("{date}")
            AND client_id in (SELECT customerId FROM MMAU_customerbase)
            ),
      MMAU
    AS (
        SELECT 
         BalanceDate,  MMAUCustomer, querycategory 
        FROM 
            ( SELECT 
            '{date}' BalanceDate, customer_id MMAUCustomer, 'Transaction' querycategory 
            FROM 
            transactional_cust
        UNION distinct
        SELECT 
           '{date}' BalanceDate, customer_id MMAUCustomer, 'Balance' querycategory 
        FROM 
            cust_with_balance)
        )
    select * from MMAU;
    
    """
    df = client.query(query).to_dataframe()
    res = pd.concat([res,df])


CPU times: total: 938 ms
Wall time: 20.2 s


In [9]:
res.groupby(['BalanceDate', 'querycategory'])['MMAUCustomer'].nunique()

BalanceDate  querycategory
2024-08-26   Balance          44947
             Transaction      35333
2024-09-02   Balance          44731
             Transaction      50555
Name: MMAUCustomer, dtype: int64

In [12]:
a = tuple(res['MMAUCustomer'][(res['BalanceDate']=='2024-09-02')&(res['querycategory'] == 'Transaction')])
a[0:5]

('1303533', '1181737', '2394592', '1685623', '1174883')

In [26]:
sq = f"""select * from  `prj-prod-dataplatform.risk_mart.customer_balance`
where clearedbalance > 1000
            AND account_type IN ("Tonik Account","Group Stash","Individual Stash")
            AND balanceDateAsOf between date_sub(date("2024-09-02"), interval 30 day) and date("2024-09-02")
            AND client_id in {a};"""
dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 763c54b2-fba3-46fc-8692-736481ad5565 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [17]:
dfd.head()

,accountid,account_type,client_id,balanceDateAsOf,clearedbalance
0,60828076180044,Group Stash,2807618,2024-08-31,5000.00
1,60828075020001,Tonik Account,2807502,2024-08-31,1000.02
2,60828088150009,Tonik Account,2808815,2024-08-31,36952.74
3,60828075020019,Individual Stash,2807502,2024-08-31,1500.13
4,60827549570005,Tonik Account,2754957,2024-08-15,8000.00


In [29]:
sq = f"""select * from  `prj-prod-dataplatform.risk_mart.customer_balance`
where clearedbalance > 1000
            AND account_type IN ("Tonik Account","Group Stash","Individual Stash")
            AND balanceDateAsOf between date_sub(date("2024-08-26"), interval 30 day) and date("2024-08-26")
            AND client_id in {a};"""
dfd1 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 10326622-abe3-40b8-87b3-8e17aef786bf successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [30]:
dfd1.head()

,accountid,account_type,client_id,balanceDateAsOf,clearedbalance
0,60827853210004,Tonik Account,2785321,2024-08-23,5000.0
1,60827151740014,Group Stash,2715174,2024-08-03,1355.0
2,60827107270007,Tonik Account,2710727,2024-08-03,2000.0
3,60827089850001,Tonik Account,2708985,2024-08-03,4000.0
4,60827188080009,Tonik Account,2718808,2024-08-03,3500.0


In [23]:
dfd1['client_id'].nunique()

33215

In [28]:
dfd['client_id'].nunique()

32927

In [37]:
sq = """SELECT 
            DATE_TRUNC(transaction_date, WEEK(MONDAY)) AS week_start_date, customer_id,  COUNT(*) cnt
            from
            `prj-prod-dataplatform.risk_mart.customer_transactions`        WHERE
            transaction_date BETWEEN DATE_SUB(date("2024-09-02"), INTERVAL 3 MONTH) AND date("2024-09-02")
            GROUP BY
            customer_id, DATE_TRUNC(transaction_date, WEEK(MONDAY))
        HAVING 
            COUNT(*) >= 5
            ;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 40fcccf9-7ea4-4e10-a691-8fc085047e52 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [38]:
dfd.shape

(128855, 3)

In [39]:
sq = """SELECT 
            DATE_TRUNC(transaction_date, WEEK(MONDAY)) AS week_start_date,customer_id,  COUNT(*) cnt
            from
            `prj-prod-dataplatform.risk_mart.customer_transactions`        WHERE
            transaction_date BETWEEN DATE_SUB(date("2024-08-26"), INTERVAL 3 MONTH) AND date("2024-08-26")
            GROUP BY
            customer_id, DATE_TRUNC(transaction_date, WEEK(MONDAY)) 
        HAVING 
            COUNT(*) >= 5
            ;
"""
dfd1 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID a0cf58df-6ac8-4424-9ec6-ce898deb0126 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [36]:
dfd1.shape

(56615, 2)

In [41]:
dfd.groupby('week_start_date')['customer_id'].nunique()

week_start_date
2024-05-27     1120
2024-06-03     7238
2024-06-10     7807
2024-06-17     7108
2024-06-24     8335
2024-07-01     8370
2024-07-08     8118
2024-07-15     7931
2024-07-22     6959
2024-07-29     8983
2024-08-05     7202
2024-08-12     8498
2024-08-19     9541
2024-08-26    29704
2024-09-02     1941
Name: customer_id, dtype: int64

In [42]:
dfd1.groupby('week_start_date')['customer_id'].nunique()

week_start_date
2024-05-20     777
2024-05-27    7995
2024-06-03    7238
2024-06-10    7807
2024-06-17    7108
2024-06-24    8335
2024-07-01    8370
2024-07-08    8118
2024-07-15    7931
2024-07-22    6959
2024-07-29    8983
2024-08-05    7202
2024-08-12    8498
2024-08-19    9541
2024-08-26    1500
Name: customer_id, dtype: int64